In [1]:
import pandas as pd
from src.training.bert_pipeline import TrainingBertPipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  5363 non-null   object 
 2   answer            5363 non-null   object 
 3   score             5363 non-null   float64
 4   normalized_score  5363 non-null   float64
 5   multibert_length  5363 non-null   int64  
 6   dataset           5363 non-null   object 
 7   dataset_num       5363 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 335.3+ KB


In [3]:
df['dataset'].value_counts()

dataset
sag               2558
analisis_essay    2162
stita              333
cunlp              171
sci                139
Name: count, dtype: int64

In [4]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results.csv"):
    df_result = pd.read_csv("experiments/results/results.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results.csv' does not exist.")

50


In [5]:
batch_sizes = [4]
overlappings = [128]
epochs_list = [10]
learning_rates = [5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [6]:
# model = [
#     ("bert_length", "bert-base-uncased"),
#     ("indobert_length", "indobenchmark/indobert-base-p1"),
#     ("albert_length", "albert-base-v1"),
#     ("indoalbert_length", "indobenchmark/indobert-lite-base-p2"),
#     ("longformer_length", "allenai/longformer-base-4096"),
#     ("multibert_length", "google-bert/bert-base-multilingual-uncased")
# ]

In [7]:
for batch_size in batch_sizes:
    for overlapping in overlappings:
        for num_epochs in epochs_list:
            for lr in learning_rates:
                results = []
                results_epoch = []
                df_result1 = None
                # Check if the second file exists
                if os.path.exists("experiments/results/results_epoch.csv"):
                    df_result1 = pd.read_csv("experiments/results/results_epoch.csv")
                    print(max(df_result1['valid_qwk']))
                else:
                    print("File 'results_epoch.csv' does not exist.")
                config = {
                    "df": df,
                    "model_name": "google-bert/bert-base-multilingual-uncased",
                    "overlapping": overlapping,
                    "batch_size": batch_size,
                    "learning_rate": lr,
                    "epochs": num_epochs,
                    "config_id": idx,
                    "max_seq_len": 256,
                    "col_length": "multibert_length",
                    "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf")
                }

                logging.info(
                    f"Running configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}"
                )
                
                print(
                    f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}"
                )
                
                try:
                    pipeline = TrainingBertPipeline(config, results, results_epoch)
                    pipeline.run_training()

                    # Save results
                    # Dapatkan root project
                    results_path = os.path.join(ROOT_DIR, "experiments/results/results.csv")
                    results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch.csv")
                    TrainingBertPipeline.save_csv(results, results_path)
                    TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
                except Exception as e:
                    logging.error(f"Error in config_id={idx}: {str(e)}")
                    print(f"Error in config_id={idx}: {str(e)}")
                    torch.cuda.empty_cache()
                finally:
                    # Clear GPU memory after every configuration
                    del pipeline.model
                    del pipeline.tokenizer
                    del pipeline.optimizer
                    torch.cuda.empty_cache()

                idx += 1

0.9251343231432688

Running configuration: config_id=51, model_name=google-bert/bert-base-multilingual-uncased, batch_size=4, max_seq_length=256, overlapping=128, epochs=10, learning_rate=1e-05
split dataset run...
create dataset run...
max len 256
max len 256
max len 256
create dataloader run...
====== Training Epoch 1/10 ======


Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors
c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Train Loss: 0.0509, Train QWK: 0.6263, Train Pearson: 0.7393
Validation Loss: 0.0303, Validation QWK: 0.8723, Validation Pearson: 0.8890
====== Training Epoch 2/10 ======
Train Loss: 0.0279, Train QWK: 0.7322, Train Pearson: 0.8614
Validation Loss: 0.0213, Validation QWK: 0.9064, Validation Pearson: 0.9057
====== Training Epoch 3/10 ======
Train Loss: 0.0203, Train QWK: 0.7646, Train Pearson: 0.9009
Validation Loss: 0.0236, Validation QWK: 0.8965, Validation Pearson: 0.9124
====== Training Epoch 4/10 ======
Train Loss: 0.0161, Train QWK: 0.7937, Train Pearson: 0.9221
Validation Loss: 0.0187, Validation QWK: 0.9143, Validation Pearson: 0.9149
====== Training Epoch 5/10 ======
Train Loss: 0.0126, Train QWK: 0.8231, Train Pearson: 0.9396
Validation Loss: 0.0218, Validation QWK: 0.9007, Validation Pearson: 0.9077
====== Training Epoch 6/10 ======
Train Loss: 0.0103, Train QWK: 0.8365, Train Pearson: 0.9506
Validation Loss: 0.0200, Validation QWK: 0.9115, Validation Pearson: 0.9122
====== T

Token indices sequence length is longer than the specified maximum sequence length for this model (1004 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0478, Train QWK: 0.6400, Train Pearson: 0.7516
Validation Loss: 0.0448, Validation QWK: 0.8334, Validation Pearson: 0.8815
====== Training Epoch 2/10 ======
Train Loss: 0.0299, Train QWK: 0.7151, Train Pearson: 0.8516
Validation Loss: 0.0265, Validation QWK: 0.8869, Validation Pearson: 0.8987
====== Training Epoch 3/10 ======
Train Loss: 0.0244, Train QWK: 0.7325, Train Pearson: 0.8807
Validation Loss: 0.0217, Validation QWK: 0.9024, Validation Pearson: 0.9024
====== Training Epoch 4/10 ======
Train Loss: 0.0201, Train QWK: 0.7815, Train Pearson: 0.9030
Validation Loss: 0.0318, Validation QWK: 0.8576, Validation Pearson: 0.8912
====== Training Epoch 5/10 ======
Train Loss: 0.0181, Train QWK: 0.7850, Train Pearson: 0.9128
Validation Loss: 0.0304, Validation QWK: 0.8609, Validation Pearson: 0.9020
====== Training Epoch 6/10 ======
Train Loss: 0.0162, Train QWK: 0.8015, Train Pearson: 0.9221
Validation Loss: 0.0205, Validation QWK: 0.9119, Validation Pearson: 0.9133
====== T